 ## only copy the params that exist in current model (caffe-like)

In [ ]:
  # only copy the params that exist in current model (caffe-like)
new_params = model.state_dict().copy()
for name, param in new_params.items():
print name
if name in saved_state_dict and param.size() == saved_state_dict[name].size():
    new_params[name].copy_(saved_state_dict[name])
    print('copy {}'.format(name))
model.load_state_dict(new_params)

# 初始化权重

In [1]:
def initialize_weights(*models):
    for model in models:
        for module in model.modules():
            if isinstance(module, nn.Conv2d) or isinstance(module, nn.Linear):
                nn.init.kaiming_normal(module.weight)
                if module.bias is not None:
                    module.bias.data.zero_()
            elif isinstance(module, nn.BatchNorm2d):
                module.weight.data.fill_(1)
                module.bias.data.zero_()

# 上采样权重初始化

In [2]:
def get_upsampling_weight(in_channels, out_channels, kernel_size):
    factor = (kernel_size + 1) // 2
    if kernel_size % 2 == 1:
        center = factor - 1
    else:
        center = factor - 0.5
    og = np.ogrid[:kernel_size, :kernel_size]
    filt = (1 - abs(og[0] - center) / factor) * (1 - abs(og[1] - center) / factor)
    weight = np.zeros((in_channels, out_channels, kernel_size, kernel_size), dtype=np.float64)
    weight[list(range(in_channels)), list(range(out_channels)), :, :] = filt
    return torch.from_numpy(weight).float()

 Converts a state dict saved from a dataParallel module to normal module state_dict inplace:param state_dict is the loaded DataParallel model_state



In [3]:
def convert_state_dict(state_dict):
    """Converts a state dict saved from a dataParallel module to normal 
       module state_dict inplace
       :param state_dict is the loaded DataParallel model_state
    
    """
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        name = k[7:] # remove `module.`
        new_state_dict[name] = v
    return new_state_dict

# 加载权重

In [4]:
def load_weights_sequential(target,source_state):
    new_dict=OrderedDict()
    for k1,v1 in target.state_dict().items():
        for k2,v2 in source_state.items():
            if 'conv' in k1 and (k2==k1):
                new_dict[k1]=v2
            else:
                new_dict[k1]=v1